In [ ]:
import os
import h5py
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import threading

stop_processing = False

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as h5_file:
        keys = list(h5_file.keys())
        
        # Assuming the first key is for the image and the second key is for the mask
        image_key = keys[0]
        mask_key = keys[1]
        
        image = np.array(h5_file[image_key])
        mask = np.array(h5_file[mask_key])
        return image, mask, image_key, mask_key

def convert_to_uint8(image):
    # Normalize the image to the range [0, 255]
    image_min = image.min()
    image_max = image.max()
    image = (image - image_min) / (image_max - image_min) * 255
    return image.astype(np.uint8)

def save_image(image, save_path):
    img = Image.fromarray(image)
    img.save(save_path)

def display_and_save_image_and_mask(image, mask, save_dir, file_name):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.title('Brain MRI Image')
    plt.imshow(image, cmap='gray')

    plt.subplot(1, 2, 2)
    plt.title('Tumor Mask')
    plt.imshow(mask, cmap='gray')

    plt.tight_layout()
    save_path = os.path.join(save_dir, f"{file_name}.png")
    plt.savefig(save_path)
    plt.close()

def process_and_save_h5_files(source_dir, target_parent_dir, generate_reference=False):
    global stop_processing
    processed_dir = os.path.join(target_parent_dir, 'processed_h5_folder')
    target_image_dir = os.path.join(processed_dir, 'images')
    target_mask_dir = os.path.join(processed_dir, 'masks')
    reference_dir = os.path.join(processed_dir, 'reference')

    if not os.path.exists(target_image_dir):
        os.makedirs(target_image_dir)
    if not os.path.exists(target_mask_dir):
        os.makedirs(target_mask_dir)
    if generate_reference and not os.path.exists(reference_dir):
        os.makedirs(reference_dir)

    files = [os.path.join(root, file) for root, _, files in os.walk(source_dir) for file in files if file.endswith('.h5')]
    total_files = len(files)

    for idx, file_path in enumerate(files):
        if stop_processing:
            break

        image, mask, image_key, mask_key = read_h5_file(file_path)

        # Convert to uint8
        image = convert_to_uint8(image)
        mask = convert_to_uint8(mask)

        # Define save paths
        file_name = os.path.splitext(os.path.basename(file_path))[0]
        image_save_path = os.path.join(target_image_dir, f"{file_name}.png")
        mask_save_path = os.path.join(target_mask_dir, f"{file_name}.png")

        # Save the image and mask
        save_image(image, image_save_path)
        save_image(mask, mask_save_path)

        # Display and save the reference image if required
        if generate_reference:
            display_and_save_image_and_mask(image, mask, reference_dir, file_name)

        # Update progress bar
        progress_var.set((idx + 1) / total_files * 100)
        progress_label.config(text=f"Processing {file_name}.h5")
        root.update_idletasks()

    if not stop_processing:
        messagebox.showinfo("Success", f"Processing completed successfully.\nFiles saved in: {processed_dir}")
    else:
        messagebox.showinfo("Stopped", "Processing was stopped.")

    # Reset stop_processing flag
    stop_processing = False

def select_source_dir():
    source_dir = filedialog.askdirectory(title="Select Source Directory")
    source_dir_entry.delete(0, tk.END)
    source_dir_entry.insert(0, source_dir)

def select_target_dir():
    target_dir = filedialog.askdirectory(title="Select Target Directory")
    target_dir_entry.delete(0, tk.END)
    target_dir_entry.insert(0, target_dir)

def start_processing():
    global stop_processing
    stop_processing = False

    source_dir = source_dir_entry.get()
    target_dir = target_dir_entry.get()
    generate_reference = reference_var.get()

    if not source_dir or not target_dir:
        messagebox.showerror("Error", "Please select both source and target directories.")
        return

    processing_thread = threading.Thread(target=process_and_save_h5_files, args=(source_dir, target_dir, generate_reference))
    processing_thread.start()

def stop_processing_func():
    global stop_processing
    stop_processing = True

# Create the main window
root = tk.Tk()
root.title("H5 File Processor")

# Create and place the widgets
tk.Label(root, text="Source Directory:").grid(row=0, column=0, padx=10, pady=10, sticky="e")
source_dir_entry = tk.Entry(root, width=50)
source_dir_entry.grid(row=0, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=select_source_dir).grid(row=0, column=2, padx=10, pady=10)

tk.Label(root, text="Target Directory:").grid(row=1, column=0, padx=10, pady=10, sticky="e")
target_dir_entry = tk.Entry(root, width=50)
target_dir_entry.grid(row=1, column=1, padx=10, pady=10)
tk.Button(root, text="Browse", command=select_target_dir).grid(row=1, column=2, padx=10, pady=10)

reference_var = tk.BooleanVar()
tk.Checkbutton(root, text="Generate Reference Images", variable=reference_var).grid(row=2, columnspan=3, padx=10, pady=10)

tk.Button(root, text="Start Processing", command=start_processing).grid(row=3, column=0, columnspan=2, padx=10, pady=10)
tk.Button(root, text="Stop Processing", command=stop_processing_func).grid(row=3, column=2, padx=10, pady=10)

progress_var = tk.DoubleVar()
progress_bar = ttk.Progressbar(root, variable=progress_var, maximum=100)
progress_bar.grid(row=4, columnspan=3, padx=10, pady=10, sticky="ew")

progress_label = tk.Label(root, text="")
progress_label.grid(row=5, columnspan=3, padx=10, pady=10)

# Run the main loop
root.mainloop()